# Settings

In [ ]:
import openai
from openai import OpenAI

 # Q. API KEY 정보를 불러와주세요
import os
from dotenv import load_dotenv
load_dotenv('./credentials/.env') 
openai.api_key = os.environ.get("OPENAI_API_KEY")
api_key = openai.api_key

import json
import time


# Q. OpenAI API를 사용하기 위한 클라이언트 객체를 생성해주세요.



# Q. 수학 과외 선생님 역할을 하는 챗봇을 생성해주세요.



# obj의 모델을 JSON 형태로 변환한 후 출력합니다.
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

show_json(assistant)

In [ ]:
# 기억할 assistant ID를 출력합니다.

ASSISTANT_ID = assistant.id
print(f"[생성한 Assistants ID]\n{ASSISTANT_ID}")

# Few-Shot

### 기존함수 정의

In [ ]:
def wait_on_run(run, thread_id):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run


def submit_message(assistant_id, thread_id, user_message):
    client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=user_message
    )
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
    )
    return run


def get_response(thread_id):
    return client.beta.threads.messages.list(thread_id=thread_id, order="asc")


def print_message(response):
    for res in response:
        print(f"[{res.role.upper()}]\n{res.content[0].text.value}\n")


def ask(assistant_id, thread_id, user_message):
    run = submit_message(
        assistant_id,
        thread_id,
        user_message,
    )

    run = wait_on_run(run, thread_id)
    print_message(get_response(thread_id).data[-2:]) 
    return run

def create_new_thread():
    thread = client.beta.threads.create()
    return thread



### Few-shot 예제 실행

In [ ]:
# 기존/새로운 스레드를 생성합니다.
thread_id = create_new_thread().id 

user_message = """
다음 영어 문장을 한국어로 번역하세요:

영어: How are you?
한국어: 잘 지내세요?

영어: What's your name?
한국어: 당신의 이름은 무엇입니까?

영어: Where is the bathroom?
한국어:

"""

run = ask(ASSISTANT_ID, thread_id, user_message)

# Zero-shot Chain-of-Thought (CoT)

### Zero-shot CoT 예제 실행 1

In [ ]:
user_message = """
질문: 사과가 5개 있습니다. 2개를 더 샀습니다. 그 후 1개의 사과를 먹었습니다. 사과는 몇 개 남았나요?

단계적으로 생각해 봅시다:

답:
"""

run = ask(ASSISTANT_ID, thread_id, user_message)

### Zero-shot CoT 예제 실행 2

In [ ]:
user_message = """
질문: 상자에 연필이 12자루 있습니다. 4자루를 꺼냈습니다. 연필은 몇 자루 남았나요?

단계적으로 생각해 봅시다:

답:
"""

run = ask(ASSISTANT_ID, thread_id, user_message)